In [3]:
import numpy as np
import os
import math
import cv2 
import matplotlib.pyplot as plt

In [46]:
class findFundamentalMat():
    
    def __init__(self, image1, image2):
        
        self.image1 = image1
        self.image2 = image2
        self.F = self.computeFMatrix()
    
    @property
    def getFMatrix(self):
        #return Fundamental Matrix
        return self.F
    
    def normalize(self, image):
        
        x , y = image[:,0], image[:,1]
        x_mean, y_mean = np.mean(x), np.mean(y)
        n = len(x)
        rms = np.sum(np.sqrt((x - x_mean)**2 + (y - y_mean)**2))
        rms = rms / n
        
        s = np.sqrt(2) / rms
        t_x = -s * x_mean
        t_y = -s * y_mean
        
        # similarity transform 
        T = np.array([[s, 0, t_x],
                    [0, s, t_y], 
                    [0, 0, 1]])
        
        n_image = list()
        for [i, j] in image:
            n_image.append(T.dot(np.array([i,j,1])))
        
        n_image = np.array(n_image)
        return T, n_image

    def computeFMatrix(self):
        
        #normalize image points
        T1, image1 = self.normalize(self.image1)
        T2, image2 = self.normalize(self.image2)
        
        # Af = 0, compute A
        A = list()
        for i,j in zip(image1, image2):
            x1 , y1 = i[0] , i[1]
            x2 , y2 = j[0] , j[1]
            # A = [x1 * x2, y1 * x2, x2, x1 * y2, y1 * y2, y2, x1, y1, 1 ]
            A.append([ x1 * x2, y1 * x2, x2, x1 * y2, y1 * y2, y2, x1, y1, 1 ])
        
        #compute f by taking SVD of A
        A = np.array(A)
        _, _ , vh = np.linalg.svd(A)
        f = vh.T #last column of v gives f
        F = f[:,-1]
        F = np.reshape(F, (3,3))
        F = F / F[-1][-1]
        
        #replace F by F'
        u , s, vh = np.linalg.svd(F)
        assert(s[0] >= s[1] >= s[2])
        s[2] = 0.0
        S = np.zeros((len(s), len(s)))
        np.fill_diagonal(S, s)
        F_ = u.dot(S.dot(vh))
        F_ = F_ / F_[-1,-1]
        
        #Denormalization F = T2^T * F' * T1
        F = T2.T.dot(F_.dot(T1))
        F = F / F[-1,-1]
        
        return F            

In [45]:
# Read and Process the .txt file
images = {0:list(),1:list()}
with open(os.getcwd()+'/FMatrixCorrPointsStereo.txt') as filehandle:
    filecontents = filehandle.readlines()
    i = 0
    j = 0
    no_of_columns = 2
    no_of_rows = round(len(filecontents) / 2)
    for line in filecontents:
        if i >= 4 and line != "\n":
            images[j].append([float(line[8:16]),float(line[24:32])])
            if i == 64:
                j = (j + 1) % no_of_columns
        i = (i + 1) % no_of_rows
    images[0] = np.array(images[0])
    images[1] = np.array(images[1])

    
inlierPts1 = images[0]
inlierPts2 = images[1]
fNorm8Point = findFundamentalMat(inlierPts1, inlierPts2)
F = fNorm8Point.getFMatrix
print(F)

# f_ = cv2.findFundamentalMat(inlierPts1, inlierPts2,cv2.FM_LMEDS)
# print(f_)

[[  3.21676174e-08  -8.34216946e-05   3.30908275e-02]
 [  8.59344122e-05   9.00008104e-06   9.09321932e-01]
 [ -3.37481007e-02  -9.15742132e-01   1.00000000e+00]]
